In [ ]:
# Transformer 모델 구축 - Transformer 대화형 챗봇(Dialogue Chatbot) 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용
# - 1. 데이터셋 로드 및 데이터 분리
# - AI Hub 대화데이터: 한국어 SNS 멀티턴 대화 데이터
# - 데이터 분리: train, validation
# - 정상 파일 추출
# - 2. JSON 파일 직접 파싱
# - 3. 사전 토크나이징 및 저장
# - 4. Dataset 클래스 정의
# - 5. 사전 토크나이징 불러오기(이전대화/현재대화) 및 DataLoader 생성
# - 6. 모델 정의
# - Feature Extraction + LoRA Fine-tuning 조합
# - 최적화 설정: optimizer, GradScaler, autocast
# - Early Stopping 클래스 정의
# - 최적 모델 가중치 저장
# - 7. 학습/검증 루프
# - 딕셔너리 형태 학습데이터를 그대로 모델에 전달하는 코드로 정리, 코드가 깔끔하고 범용적으로 사용한다
# - Early Stopping 객체 사용하여 적용
# - AMP torch.float32 사용(메모리 사용 증가, torch.float16 사용시 loss가 너무 작아저 nan 발생)
# - 8. 전체 평가 파이프라인
# - F1/EM 평가
# - 9. 모델 정의 및 최적화 모델 로드
# - 10. 멀티 답변 생성
# - 11. 문장 추론: Fast API 호출

In [1]:
# 데이터셋 로드 
# - AI Hub 대화데이터: 한국어 SNS 멀티턴 대화 데이터
# - 데이터 분리: train, validation
# - 정상 파일 추출
import glob, random, json

# 참고: AI Hub에서 제공하는 대화데이터의 JSON 구조가 불안하여 전체 로드시 파싱 에러 발생
# - 해결책: JSON 파일 직접 파싱
train_files = glob.glob('llm_data/ai_hub_dialogue_session2/train/*.json')
valid_files = glob.glob('llm_data/ai_hub_dialogue_session2/validation/*.json')

# 10% 샘플링
random.seed(42)
sample_train = random.sample(train_files, int(len(train_files) * 0.1))
sample_valid = random.sample(valid_files, int(len(valid_files) * 0.1))

def validate_file(file):
    try:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 최소한 sessionInfo와 dialog가 있어야 정상 파일로 간주
        if 'sessionInfo' in data:
            for session in data['sessionInfo']:
                if 'dialog' in session:
                    return True
        return False
    except Exception:
        return False

# 정상 파일만 추출
valid_train_files = [ f for f in sample_train if validate_file(f) ]
valid_valid_files = [ f for f in sample_valid if validate_file(f) ]

print(len(valid_train_files))
print(len(valid_valid_files))

3308
413


In [2]:
# JSON 파일 직접 파싱
import json

def load_aihub(files):
    contexts, responses = [], []
    for file in files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f) # JSON 파일 로드
            for session in data.get('sessionInfo', []): # sessionInfo 리스트 로드
                dialog = session.get('dialog', []) # dialog 리스트 로드
                for i in range(1, len(dialog)): # dialog 리스트 갯수, i는 1부터 시작
                    contexts.append(dialog[i-1].get('utterance', '')) # 이전 발화, 0번째 인덱스 데이터
                    responses.append(dialog[i].get('utterance', '')) # 현재 발화, 1번째 인덱스 데이터
        except Exception as e:
            print(f'파일 {file} 처리 실패: {e}')
    # return {'context': contexts, 'response': responses}
    return contexts, responses # 리스트 반환

# 예시: AIHub 데이터 로드
train_contexts, train_responses = load_aihub(valid_train_files)
valid_contexts, valid_responses = load_aihub(valid_valid_files)

In [3]:
# 사전 토크나이징 및 저장
import torch
from transformers import AutoTokenizer

# google/mt5-small 한국어 포함 다국어 처리가 가능한 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", legacy=False)

# 토크나이징 및 저장 함수
# - contexts 대화 맥락 리스트, responses 응답 리스트, tokenizer 토크나이저, max_length 최대 토큰 길이 기본 64, save_path 저장 경로
def preprocess_and_save(contexts, responses, tokenizer, max_length=64, save_path="./llm_models/21_transformer_dialogue_chatbot/train_data.pt"):
    input_ids_list, attention_masks_list, labels_list = [], [], []

    for context, response in zip(contexts, responses):
        # 토크나이징: context, response
        inputs = tokenizer( # 결과 shape: (1, max_length) (1, 64)
            context,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        labels = tokenizer(
            response,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )["input_ids"]

        # lables에서 PAD 토큰을 -100으로 치환
        labels[labels == tokenizer.pad_token_id] = -100

        input_ids_list.append(inputs["input_ids"].squeeze(0)) # inputs["input_ids"] (1,64) -> inputs["input_ids"].squeeze(0) (64)
        attention_masks_list.append(inputs["attention_mask"].squeeze(0)) # inputs["attention_mask"] (1,64) -> inputs["attention_mask"].squeeze(0) (64)
        labels_list.append(labels.squeeze(0)) # labels (1,64) -> labels.squeeze(0) (64)

    # 학습에 필요한 모든 데이터를 포함한 딕셔너리 형태
    dataset_tensors = {
        "input_ids": torch.stack(input_ids_list), # shape (대화쌍이 1000개라면,64) (1000,64)
        "attention_mask": torch.stack(attention_masks_list), # shape (대화쌍이 1000개라면,64) (1000,64)
        "labels": torch.stack(labels_list) # shape (대화쌍이 1000개라면,64) (1000,64)
    }
    torch.save(dataset_tensors, save_path)
    print(f"Saved pre-tokenized dataset at {save_path}")

# 학습 전에 한 번만 실행
preprocess_and_save(train_contexts, train_responses, tokenizer, save_path="./llm_models/21_transformer_dialogue_chatbot/train_data.pt")
preprocess_and_save(valid_contexts, valid_responses, tokenizer, save_path="./llm_models/21_transformer_dialogue_chatbot/valid_data.pt")

Saved pre-tokenized dataset at ./llm_models/21_transformer_dialogue_chatbot/train_data.pt
Saved pre-tokenized dataset at ./llm_models/21_transformer_dialogue_chatbot/valid_data.pt


In [4]:
# Dataset 클래스 정의
from torch.utils.data import Dataset

class MyDataset(Dataset):
    # dataset_tensors 딕셔너리: "input_ids", "attention_mask", "labels" 세 가지 텐서를 포함한 구조
    def __init__(self, data):
        self.data = data
    
    # "input_ids"의 첫번째 차원(샘플 개수)을 기준으로 길이를 계산
    def __len__(self):
        return len(self.data["input_ids"]) # (1000,64) -> 1000 길이 리턴
    
    # idx(정수 인덱스) 값에 맞는 샘플 하나를 반환하는 역할
    # - 예시 1000 안에서의 인덱스 순번이다
    def __getitem__(self, idx):
        return {
            "input_ids": self.data["input_ids"][idx],
            "attention_mask": self.data["attention_mask"][idx],
            "labels": self.data["labels"][idx],
        }

In [5]:
# 사전 토큰나이징 파일 불러오기 및 DataLoader 생성
from torch.utils.data import DataLoader

# 사전 토큰나이징 파일 불러오기
train_data = torch.load("./llm_models/21_transformer_dialogue_chatbot/train_data.pt")
valid_data = torch.load("./llm_models/21_transformer_dialogue_chatbot/valid_data.pt")

# Dataset 객체 생성
train_dataset = MyDataset(train_data)
valid_dataset = MyDataset(valid_data)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

In [6]:
# 모델 정의
# - 모델 본체 동결 처리: Feature Extraction
# - LoRA 파인튜닝 적용
# - Early Stopping 적용
import torch
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model
from torch.amp import GradScaler, autocast
import os

# - 구글의 mT5-small 모델: 다국어 Seq2Seq 언어모델로, 입력(context)을 받아 응답(response)을 생성하는 데 적합하다
model = AutoModelForSeq2SeqLM.from_pretrained('google/mt5-small', tie_word_embeddings=False, use_safetensors=True)

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# 모델 전체를 GPU/CPU 디바이스 메모리로 이동
model = model.to(device)

# 모델 본체 동결 처리: Feature Extraction
# - 원래 mT5 모델의 모든 파라미터를 **동결(freeze)** 처리, 즉 기존 모델은 그대로 두고 학습하지 않는다
# - Feature Extractor로만 사용되고, 추가 모듈인 LoRA 모듈만 학습하는 구조가 된다
for param in model.parameters():
    param.requires_grad=False

# LoRA 적용: LoRA 모듈만 학습되도록 설정(경량 파인튜닝)
lora_config = LoraConfig(
    r=8, # 작은 rank(r=8)로 효율적인 파인튜닝 가능
    lora_alpha=32, # LoRA scaling factor
    target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    lora_dropout=0.1, # 드롭아웃
    bias='none',
    task_type='SEQ_2_SEQ_LM' # 대화 응답 생성은 Seq2Seq LM
)
# LoRA 모델 생성
model = get_peft_model(model, lora_config)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
# Automatic Mixed Precision(AMP) 학습을 위한 GradScaler 준비
# Automatic Mixed Precision(AMP)은 모델 파라미터는 FP32로 유지하면서 연산(곱셈·덧셈 등)만 FP16으로 자동 전환하여, 
# 정밀도 손실 없이 메모리와 연산 효율을 극대화하는 기술
scaler = GradScaler()
num_epochs = 3

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0, path='./llm_models/21_transformer_dialogue_chatbot/best_model.pt'):
        self.patience=patience
        self.min_delta=min_delta
        self.best_loss=None
        self.counter=0
        self.early_stop=False
        self.path=path
    
    def __call__(self, valid_loss, model):
        # 최조 손실값에 해당하는 모델 가중치 저장
        if self.best_loss is None:
            self.best_loss=valid_loss
            self.save_checkpoint(model)
        
        # 성능 개선 -> 최적 모델 갱신
        elif valid_loss < self.best_loss - self.min_delta:
            self.best_loss=valid_loss
            self.counter=0
            self.save_checkpoint(model)
        
        # 개선 없음
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop=True
    
    def save_checkpoint(self, model):
        # 디렉토리만 생성
        folder = os.path.dirname(self.path)
        if folder !='' and not os.path.exists(folder):
            os.makedirs(folder)
        # 모델 가중치 저장
        torch.save(model.state_dict(), self.path)
        print(f' Best model saved at {self.path}')

# Early Stopping 객체 생성
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

# 모델 확인
print(model)

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

MT5ForConditionalGeneration LOAD REPORT from: google/mt5-small
Key                         | Status  | 
----------------------------+---------+-
shared.weight               | MISSING | 
encoder.embed_tokens.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


PyTorch Version: 2.7.1+cu118, Device: cuda
PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 512)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 512)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=384, bias=False)
                    )
                    (

In [ ]:
# - 학습/검증 루프
# - 딕셔너리 형태 학습데이터를 그대로 모델에 전달하는 코드로 정리, 코드가 깔끔하고 범용적으로 사용한다
# - Early Stopping 객체 사용하여 적용
# - AMP torch.float32 사용(메모리 사용 증가, torch.float16 사용시 loss가 너무 작아저 nan 발생)

from tqdm import tqdm

for epoch in range(num_epochs):
    # Train
    model.train() # 학습 모드 지정
    total_train_loss = 0

    # Train Loop
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]'):
        batch = { k: v.to(device) for k, v in batch.items() } # 딕셔너리 형태로 생성, 학습데이터 GPU 지정
        # input_ids, attention_mask, labels = [x.to(device) for x in batch] # 리스트 언패킹

        optimizer.zero_grad() # 오차역전파 코드, 미분 전 가중치/바이어스 파라미터 초기화

        # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
        # with autocast(device_type='cuda', dtype=torch.float16):
        
        # RTX 30 시리즈 GPU는 BF16을 지원하기 때문에, 안정성과 속도를 동시에 확보할 수 있다
        with autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model(**batch) # 딕셔너리 형태 학습데이터를 그대로 모델에 전달, 코드가 깔끔하고 범용적으로 사용한다
            loss = outputs.loss # 손실값
        scaler.scale(loss).backward() # 미분 연산
        scaler.step(optimizer) # 미분 연산 후 가중치/바이어스 파라미터 업데이트
        scaler.update()

        # with autocast(device_type='cuda', enabled=False):
        #     outputs = model(**batch)
        #     # outputs = model(input_ids=input_ids,
        #     #         attention_mask=attention_mask,
        #     #         labels=labels)

        #     loss = outputs.loss
        # loss.backward()
        # optimizer.step()

        total_train_loss += loss.item() # 손실 누적
    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}')

    # Validation Loop
    model.eval() # 검증/추론 모드 지정
    total_val_loss = 0

    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f'Epoch {epoch+1} [Valid]'):
            batch = { k: v.to(device) for k, v in batch.items() }
            # input_ids, attention_mask, labels = [x.to(device) for x in batch] # 리스트 언패킹

            # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
            # with autocast(device_type='cuda', dtype=torch.float16):

            # RTX 30 시리즈 GPU는 BF16을 지원하기 때문에, 안정성과 속도를 동시에 확보할 수 있다
            with autocast(device_type='cuda', dtype=torch.bfloat16):
                outputs = model(**batch) # 검증 모델 예측
                loss = outputs.loss # 검증 손실값

            # with autocast(device_type='cuda', enabled=False):
            #     outputs = model(**batch)
            #     # outputs = model(input_ids=input_ids,
            #     #     attention_mask=attention_mask,
            #     #     labels=labels)

            #     loss = outputs.loss

            total_val_loss += loss.item() # 검증 손실값 누적    
    avg_val_loss = total_val_loss / len(valid_loader)
    print(f'Epoch {epoch+1}, Valid Loss: {avg_val_loss:.4f}')

    # Early Stopping 체크
    early_stopping(valid_loss=avg_val_loss, model=model)
    if early_stopping.early_stop:
        print('Early stopping triggered.')
        break

In [ ]:
# 전체 평가 파이프라인: F1/EM 평가


In [ ]:
# 모델 정의 및 최적화 모델 로드

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# 같은 구조의 모델 초기화
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small', legacy=False)
model = AutoModelForSeq2SeqLM.from_pretrained('google/mt5-small') # MT5 모델 불러온다

# 모델 본체 동결 처리: Feature Extraction
for param in model.parameters():
    param.requires_grad=False
        
# LoRA 적용: LoRA 모듈만 학습되도록 설정(경량 파인튜닝)
lora_config = LoraConfig(
    r=8, # 작은 rank(r=8)로 효율적인 파인튜닝 가능
    lora_alpha=32, # LoRA scaling factor
    # target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    lora_dropout=0.1, # 드롭아웃
    bias='none',
    task_type='SEQ_2_SEQ_LM' # 대화 응답 생성은 Seq2Seq LM
)
# LoRA 모델 생성
model = get_peft_model(model, lora_config)

# 저장된 state_dict 불러오기
model.load_state_dict(torch.load('./llm_models/21_transformer_dialogue_chatbot/best_model.pt'))

# 추론/검증 모드 적용
model.eval()

# 모델 전체를 GPU/CPU 디바이스 메모리로 이동
model = model.to(device)

# 모델 확인
print(model)

In [ ]:
# 멀티 답변 생성

import re

# context = "서울에서 유명한 관광지 알려줄래?"
context = "서울의 유명 관광지는 경복궁, 남산타워, 명동입니다."

inputs = tokenizer(
    context,
    return_tensors='pt'
).to(device)

# Beam search + Sampling 혼합
outputs = model.generate(
    **inputs,
    max_length=128,
    min_length=8, # 최소 길이 강제

    num_beams=5, # 5~7 정도가 대화형 모델에서는 가장 균형이 좋다
    early_stopping=True,
    do_sample=True,
    top_p=0.85, # 0.9 → 0.6로 줄여서 불필요한 변형을 줄인다, 변형 단어 출력 감소
    temperature=0.8, # 0.7 → 0.5 정도로 낮추면 더 안정적인 답변을 얻을 수 있다
    num_return_sequences=3,

    repetition_penalty=2.0, # 반복 억제
    no_repeat_ngram_size=4, # 4-gram 반복 금지
    length_penalty=1.2 # 1.2~2.0 정도로 설정하면, 모델이 불필요하게 길게 반복하는 걸 줄인다
)

for i, output in enumerate(outputs):
    decoded = tokenizer.decode(output, skip_special_tokens=True)

    # <extra_id_n> 토큰 제거
    # decoded = re.sub(r'<extra_id_\d+>', '', decoded).strip()
    decoded = decoded.replace('..', '.').replace('??', '?')
    decoded = re.sub(r'북산타워|서산타워|여산타워|한산타워', '남산타워', decoded)
    # decoded = re.sub(r'^,?\s*의 유명 관광지', '서울의 유명 관광지는', decoded)
    # decoded = re.sub(r'(국립공원|남산타워)(,\s*\1)+', r'\1', decoded)  # 중복 제거
    # decoded = re.sub(r'궁금한 관광지입니다', '관광 명소로 잘 알려져 있습니다', decoded)

    print(f'답변 {i+1}: {decoded}')